In [1]:
import pandas as pd
import numpy as np


In [2]:
# read training data 
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train = df_train.drop(columns = ['Unnamed: 0', 'zipcode'])
df_test = df_test.drop(columns=['Unnamed: 0', 'zipcode', 'date', 'id'])
df_train['price'] = df_train['price']/1000
df_test['price'] = df_test['price']/1000
df_train

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
0,221.90,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,47.5112,-122.257,1340,5650
1,538.00,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,47.7210,-122.319,1690,7639
2,180.00,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,47.7379,-122.233,2720,8062
3,604.00,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,47.5208,-122.393,1360,5000
4,510.00,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,291.00,4,2.50,1860,6325,2.0,0,0,4,7,1860,0,1991,0,47.3492,-122.030,1860,6449
996,199.95,2,2.75,1590,20917,1.5,0,0,3,5,1590,0,1920,0,47.2786,-122.250,1310,6000
997,553.50,2,1.00,850,2340,1.0,0,0,3,7,850,0,1922,0,47.6707,-122.328,1300,3000
998,189.95,2,1.00,1030,4188,1.0,0,0,3,8,1030,0,1981,0,47.3738,-122.057,1450,3376


In [4]:
# select features and target
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

x_train = df_train.loc[:,df_train.columns != 'price']
y_train = df_train.loc[:, 'price']
x_test = df_test.loc[:,df_test.columns != 'price']
y_test = df_test.loc[:, 'price']

scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns=x_test.columns)

array([[ 1.        , -0.40982347, -1.44988843, ..., -0.35519332,
        -0.96563661, -0.3128578 ],
       [ 1.        , -0.40982347,  0.28318404, ..., -0.7998304 ,
        -0.44332966, -0.23355563],
       [ 1.        , -1.58410276, -1.44988843, ..., -0.18307573,
         1.09374507, -0.21669046],
       ...,
       [ 1.        , -1.58410276, -1.44988843, ..., -0.86437449,
        -1.02532883, -0.4185143 ],
       [ 1.        , -1.58410276, -1.44988843, ...,  1.07911986,
        -0.80148299, -0.40352304],
       [ 1.        , -0.40982347, -0.06343045, ..., -0.46993837,
         0.39236146, -0.15736334]])

In [20]:
#3 linear regression
from numpy.linalg import pinv
class MyLinearRegression:
    def __init__(self):
        self.theta_ = []
    
    def fit(self, X, y):
        X_arr = np.insert(X.to_numpy(), 0, 1, axis=1)
        y_arr = y.to_numpy()
        # we know theta = (X^TX)^-1X^Ty
        theta = pinv(X_arr.transpose()@X_arr)@(X_arr.transpose()@y_arr)
        self.theta_ = theta
        return theta
    
    def predict(self, X):
        X_arr = np.insert(X.to_numpy(), 0, 1, axis=1)
        return X_arr@self.theta_

In [21]:
#4 polynomial regression
class PolyRegression(MyLinearRegression):
    def __init__(self, p, feature):
        self.p = p
        self.feature = feature
        
    # Converts a dataframe to the form for polynomial regressionwith columns for each order of a given feature
    def convert_to_poly(self, x):
        x_poly = pd.DataFrame(x[self.feature].copy())
        for power in range(2, self.p + 1):
            column = self.feature + '^' + str(power)
            x_poly[column] = x[self.feature] ** power
        scaler = StandardScaler()
        x_poly = pd.DataFrame(scaler.fit_transform(x_poly), columns=x_poly.columns)
        return x_poly
    
    def fit(self, X, y):
        X_arr = self.convert_to_poly(X)
        return super().fit(X_arr, y)
    
    def predict(self, X):
        X_arr = self.convert_to_poly(X)
        return super().predict(X_arr)
    

In [22]:
#5 gradient descent
from numpy.linalg import norm
class GradientDescentLR:
    def __init__(self, alpha, epsilon = .01, max_iter=1000):
        self.alpha = alpha
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.theta_ = []
       
    # fits the model to the training data
    def fit(self, X, y):
        X_arr = np.insert(X.to_numpy(), 0, 1, axis=1)
        # Initialize random theta
        self.theta_ = np.random.rand(X_arr.shape[1])       
        N = X_arr.shape[0]
        c = self.alpha * 2 / N # constant during the iterating
        for i in range(0, self.max_iter):
            # theta -= alpha(2/N)(X*theta - y)^T * X
            theta_new = self.theta_ - c * (X_arr @ self.theta_ - y).transpose() @ X_arr
            delta = np.linalg.norm(theta_new - self.theta_)
            self.theta_ = theta_new
            if delta < self.epsilon:
                break
        # Save instance variables used by run_model method
        self.coef_ = self.theta_[1:]
        self.intercept_ = self.theta_[0]
           
    # Predict Y values given X values using model fit
    def predict(self, X):
        X2 = np.insert(X.to_numpy(), 0, 1, axis=1)
        result = X2 @ self.theta_
        return result

In [23]:
# ridge
class RidgeRegression:
    def __init__(self, lam, alpha, max_iters):
        self.lam = lam
        self.alpha = alpha
        self.max_iters = max_iters
        self.epsilon = .1
        self.coef_ = []
        self.theta_ = []
    
    def fit(self, X, y):
        X_arr = np.insert(X.to_numpy(), 0, 1, axis=1)
        X_t = X_arr.transpose()
        c0 = 2 * self.alpha
        c = 1 - 2 * self.alpha * self.lam
        scale = c * np.identity(X_arr.shape[1])  # apply to I so we can multiply against theta
        scale[0, 0] = 1  # no regularization for theta[0]
        self.theta_ = np.random.rand(X_arr.shape[1])
        for i in range(0, max_iter):
            theta_new = scale @ self.theta_ - c0 * X_t @ (X_arr @ self.theta_ - y)
            delta = np.linalg.norm(theta_new - self.theta_)
            self.theta_ = theta_new
            if (delta < self.epsilon):
                break
        self.coef_ = self.theta_[1:]
        self.intercept_ = self.theta_[0]
        return self.theta_
    
    def predict(self, X):
        X_arr = np.insert(X.to_numpy(), 0, 1, axis=1)
        result = X_arr @ self.theta_ 
        return result.flatten()


In [24]:
# Common function that is used for all models in this assigment
# Fits the data, produces model performance on training and testing data
# Supports all models in this assignment
def run_model(X_train, Y_train, X_test, Y_test, model, print_coef=False):
    model.fit(X_train, Y_train)
    if print_coef:
        print(f'Intercept: {model.intercept_:.2f}')
        print('Coefficients\n', model.coef_)

    Y_train_predicted = model.predict(X_train)
    mse = mean_squared_error(Y_train, Y_train_predicted)
    r2= r2_score(Y_train, Y_train_predicted)

    print("Training set performance")
    print(f'   MSE = {format_nbr(mse)}')
    print(f'   R2 = {format_nbr(r2)}')
   
    Y_test_predicted = model.predict(X_test)
    mse = mean_squared_error(Y_test, Y_test_predicted)
    r2= r2_score(Y_test, Y_test_predicted)

    print("Testing set performance")
    print(f'   MSE = {format_nbr(mse)}')
    print(f'   R2 = {format_nbr(r2)}')
    
# format a number based on its magnitude
# If <= 100000, print using :.3f
# Else print using :.0e
def format_nbr(f):
    if abs(f) < 100000:
        return f'{f:.3f}'
    else:
        return f'{f:.4e}'

In [25]:
#2. run sklearn linear regression
run_model(x_train, y_train, x_test, y_test, LinearRegression(), print_coef=True)
print(x_train.columns)

Intercept: 520.41
Coefficients
 [-12.52196187  18.52763251  56.7488368   10.88186845   8.04372084
  63.74289956  48.20010852  12.96426936  92.23147482  48.29008886
  27.13703247 -67.64311741  17.27137953  78.37573693  -1.03520308
  45.57765781 -12.93009098]
Training set performance
   MSE = 31486.168
   R2 = 0.727
Testing set performance
   MSE = 59784.366
   R2 = 0.641
Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')


In [26]:
#3. run home grown linear regression
run_model(x_train, y_train, x_test, y_test, MyLinearRegression())

Training set performance
   MSE = 31486.168
   R2 = 0.727
Testing set performance
   MSE = 59784.366
   R2 = 0.641


In [27]:
#4. run polynomial regression with varying degrees for p
for p in range(1,6):
    print ('**** degree=', p, "feature=", "sqft_living")
    run_model(x_train, y_train, x_test, y_test, PolyRegression(p, 'sqft_living'))

**** degree= 1 feature= sqft_living
Training set performance
   MSE = 57947.526
   R2 = 0.497
Testing set performance
   MSE = 89911.924
   R2 = 0.461
**** degree= 2 feature= sqft_living
Training set performance
   MSE = 54822.665
   R2 = 0.524
Testing set performance
   MSE = 77663.306
   R2 = 0.534
**** degree= 3 feature= sqft_living
Training set performance
   MSE = 53785.195
   R2 = 0.533
Testing set performance
   MSE = 83389.238
   R2 = 0.500
**** degree= 4 feature= sqft_living
Training set performance
   MSE = 52795.775
   R2 = 0.541
Testing set performance
   MSE = 85959.973
   R2 = 0.484
**** degree= 5 feature= sqft_living
Training set performance
   MSE = 52626.112
   R2 = 0.543
Testing set performance
   MSE = 86754.834
   R2 = 0.480


In [28]:
#5. run gradient descent with varying alphas, and max iterations
for alpha in [0.01, 0.1, 0.5]:
    for max_iter in [10, 50, 100]:
        print('**** alpha =', alpha, "max_iter =", max_iter)
        run_model(x_train, y_train, x_test, y_test, GradientDescentLR(alpha=alpha, max_iter = max_iter), print_coef=False)

**** alpha = 0.01 max_iter = 10
Training set performance
   MSE = 2.3476e+05
   R2 = -1.039
Testing set performance
   MSE = 2.9208e+05
   R2 = -0.752
**** alpha = 0.01 max_iter = 50
Training set performance
   MSE = 69650.280
   R2 = 0.395
Testing set performance
   MSE = 1.0786e+05
   R2 = 0.353
**** alpha = 0.01 max_iter = 100
Training set performance
   MSE = 36822.842
   R2 = 0.680
Testing set performance
   MSE = 68913.242
   R2 = 0.587
**** alpha = 0.1 max_iter = 10
Training set performance
   MSE = 35100.876
   R2 = 0.695
Testing set performance
   MSE = 66551.763
   R2 = 0.601
**** alpha = 0.1 max_iter = 50
Training set performance
   MSE = 31497.388
   R2 = 0.726
Testing set performance
   MSE = 59905.389
   R2 = 0.641
**** alpha = 0.1 max_iter = 100
Training set performance
   MSE = 31486.431
   R2 = 0.727
Testing set performance
   MSE = 59802.474
   R2 = 0.641
**** alpha = 0.5 max_iter = 10
Training set performance
   MSE = 1.4059e+17
   R2 = -1.2211e+12
Testing set perfor

In [29]:
#6. create set of random data based on Y = 2X + 1 + e
x_train_rand = pd.DataFrame(np.random.uniform(low=-2.0, high=2.0, size=(1000,1)), columns=['X'])
y_train_rand = 1 + 2 * x_train_rand['X'] + np.random.normal(0, 2, 1000)
x_test_rand = pd.DataFrame(np.random.uniform(low=-2.0, high=2.0, size=(1000,1)), columns=['X'])
y_test_rand = 1 + 2 * x_test_rand['X'] + np.random.normal(0, 2, 1000)

In [30]:
#6. run linear regression on random data
run_model(x_train_rand, y_train_rand, x_test_rand, y_test_rand, LinearRegression())

Training set performance
   MSE = 4.160
   R2 = 0.549
Testing set performance
   MSE = 4.071
   R2 = 0.585


In [31]:
#6. run ridge regression on random data
for l in [0.1, 1, 10, 100, 1000, 10000]:
    alpha = 0.00007
    print(f'Ridge Regression lambda={l}, alpha={alpha},  max_iter=100')
    run_model(x_train_rand, y_train_rand, x_test_rand, y_test_rand, RidgeRegression(l, alpha, 100), print_coef=True)

Ridge Regression lambda=0.1, alpha=7e-05,  max_iter=100
Intercept: 0.71
Coefficients
 [1.66798156]
Training set performance
   MSE = 4.368
   R2 = 0.526
Testing set performance
   MSE = 4.331
   R2 = 0.558
Ridge Regression lambda=1, alpha=7e-05,  max_iter=100
Intercept: 0.99
Coefficients
 [1.52606619]
Training set performance
   MSE = 4.420
   R2 = 0.520
Testing set performance
   MSE = 4.487
   R2 = 0.542
Ridge Regression lambda=10, alpha=7e-05,  max_iter=100
Intercept: 0.73
Coefficients
 [1.61057808]
Training set performance
   MSE = 4.408
   R2 = 0.522
Testing set performance
   MSE = 4.396
   R2 = 0.552
Ridge Regression lambda=100, alpha=7e-05,  max_iter=100
Intercept: 0.65
Coefficients
 [1.54750764]
Training set performance
   MSE = 4.521
   R2 = 0.509
Testing set performance
   MSE = 4.519
   R2 = 0.539
Ridge Regression lambda=1000, alpha=7e-05,  max_iter=100
Intercept: 0.56
Coefficients
 [0.98781225]
Training set performance
   MSE = 5.624
   R2 = 0.390
Testing set performance
 